# Definition of a conda environment to work with SIXTE + python + xspec:
# ==================================================================

        >conda create --name tutorial python  
        >conda activate tutorial    
        >conda install matplotlib jupyterlab pandas astropy
        >conda install -c conda-forge gxx_linux-64==11.1.0  # compatibility issues with xspec/heasoft  
        >conda install krb5 curl  


# 10.2 General Introduction to Simulations with SIXTE

As discussed in more detail in Sect. 1, SIXTE is a Monte Carlo based tool that can simulate observations of
astrophysical sources for a wide variety of different current and future X-ray astronomical satellites. A SIXTE
simulation consists of three steps:  
   1. Preparation of the input of the simulation, using so-called SIMPUT files (Sect. 2): In this step we define what is to be observed. Typically this will be a pointed observation of a field on the sky. In the most simple case, this field will only contain one point source, but the simulations also allow to take into account extended sources, time variability, or the simulation of large catalogues of astronomical sources that can contain millions of X-ray sources on the whole sky. Later examples in this and the following sections will present some of the more advanced features.  
   2. Running the simulation: In this step photons from all sources that are visible to the instrument are generated using a Monte Carlo algorithm (Sect. 3). These photons are then projected onto the focal plane of the instrument using a model of the instrumental optics (Sect. 4) where they are detected using one of the available instrumental models (Sect. 6).  
   3. Analyzing the simulation: The output of the previous step are one or multiple standard FITS event files whose structure follows X-ray astronomical standards as specified by NASA’s HEASARC. The output can therefore be analyzed using standard astronomical data analysis packages. SIXTE provides some tools that prepare standard data products such as spectra and images from event files, alternatively other tools that can read FITS event files can be used.  
   
Before we start in the next sections with more complicated examples, in the following we will illustrate how a
SIXTE simulation works using a simple observation of a point source.


In [ ]:
# import some general packages
import copy
import glob
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas
import time
import tempfile
import shutil, shlex
from datetime import datetime
from subprocess import check_call, STDOUT
from funcs import *

# Astropy
from astropy.io import fits
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)

#XSPEC
from xspec import Xset, Plot, AllData, ModelManager, Spectrum, Model, AllModels, Fit

In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]
base = "mcrab"
xmldir = f"{SIXTE}/share/sixte/instruments/athena-wfi/wfi_wo_filter_B4C"

## 10.2.1 Step 0: SIXTE installation setup
## 10.2.2 Step 1: Preparing the SIMPUT file


### Create an xspec model and save it as mcrab.xcm

Power law spectrum: $\Gamma=2.05$ 

Unabsorbed Flux(2-10keV): $2.16 \times 10^{-11} \rm{erg\,cm^{-2}\,s^{-1}}$  

Foregroung absorption: $N_H=2\times 10^{21} \rm{cm^{-2}}$  

XSPEC model: phabs*pegpwrlw

In [ ]:
# Example of a xspec file with the model "phabs*pegpwrlw"
AllModels.clear()
base = "mcrab"
xcm = f"{base}.xcm"
# define XSPEC parameters
Xset.abund = "wilm"
Xset.cosmo = "70 0. 0.73"
Xset.xsect = "bcmc"
mcmod = Model("phabs*pegpwrlw")
mcmod.phabs.nH = 0.2
mcmod.pegpwrlw.PhoIndex = 2.05
mcmod.pegpwrlw.eMin = 2.
mcmod.pegpwrlw.eMax = 10.
mcmod.pegpwrlw.norm = 21.6
mcmod.show()

In [ ]:
# Get FLUX value to use in SIMPUTFILE run
AllModels.calcFlux("2. 10.")

In [ ]:
simput_flux = AllModels(1).flux[0]
print("Simput Flux=", simput_flux)

In [ ]:
clobber = True
# If clobber is true and the file exists, it will be removed before creating a new one.
if os.path.exists(xcm):
    if clobber:
        os.remove(xcm)
    else:
        raise Exception(f"Xspec file ({xcm}) already exists: it will not be overwritten") 
Xset.save(xcm)

### <span style="color:blue">$\textbf{Exercise}$</span>:
#### Add a narrow Gaussian iron line at an energy of 6.4 keV with an equivalent width of 150 eV to the spectrum.

In [ ]:
AllModels.clear()
AllData.clear()
mcmodG = Model("phabs*pegpwrlw + ga")
mcmodG.phabs.nH = 0.2
mcmodG.pegpwrlw.PhoIndex = 2.05
mcmodG.pegpwrlw.eMin = 2.
mcmodG.pegpwrlw.eMax = 10.
mcmodG.pegpwrlw.norm = 21.6
mcmodG.gaussian.LineE = 6.4
mcmodG.gaussian.Sigma = 0.001
mcmodG.gaussian.norm = 3.01e-5 ## modify this value as needed to get the 150eV equivalent width
mcmodG.show()
AllModels.eqwidth(3) # value is only stored if spectrum is loaded (which is not)


### Create SIMPUT FILE for source (``simputfile``)

In [ ]:
simputfile = f"{base}.fits"
xcmfile = f"{base}.xcm"

comm = (f"simputfile Simput={simputfile} Src_Name=first RA=0. Dec=0. srcFlux={simput_flux} Elow=0.1 Eup=15. " +
        f"NBins=1000 Emin=2 Emax=10. XSPECFile={xcmfile} clobber=yes")
#print(comm)
run_comm(comm, "Creating simputfile")

### <span style="color:blue"> $\textbf{Exercise}$</span>: 
#### Generate a second SIMPUT file for our source, using the plPhoIndex, plFlux, and NH parameters rather than an xcm-file. Compare both SIMPUT-files using fv. **Note this step only works if you have ISIS installed**


In [ ]:
simputfile = f"{base}_wo-xcm.fits"

comm = (f"simputfile Simput={simputfile} Src_Name=first RA=0. Dec=0. srcFlux={simput_flux} Elow=0.1 Eup=15. " +
        f"Emin=2 Emax=10. clobber=yes plPhoIndex=2.05 plFlux=21.6 NH=0.2 ")
#print(comm)
run_comm(comm, "Creating simputfile")

## 10.2.3 An aside: Inspecting FITS-files with (HEASOFT)/Astropy tools


In [ ]:
simputfile = f"{base}.fits"
f = fits.open(simputfile)
f.info()
f[1].columns.names
f.close()

## 10.2.4 Step2: Running the simulation

### Simulate with WFI one large chip (``runsixt``)

In [ ]:
xmldir = f"{SIXTE}/share/sixte/instruments/athena-wfi/wfi_wo_filter_B4C"
xml = f"{xmldir}/ld_wfi_ff_large.xml"
evtfile = f"sim_evt_{base}.fits"

comm = (f"runsixt XMLFile={xml} RA=0. Dec=0. Prefix='' Simput={simputfile} EvtFile={evtfile} Exposure=1000 clobber=yes")
#print(comm)
run_comm(comm, "Simulating source with  WFI  instrument")

### <span style="color:blue">$\textbf{Exercise}$</span>: 
#### Modify your SIMPUT-file to cover the full energy range of the ARF. 
##### Note that you actually have to go slightly past the limits listed in the ARF because the energy grid of the spectrum is generated at the midpoints of the bins from Elow to Eup.


In [ ]:
# New simputfile
simputfile = f"{base}.fits"
xcmfile = f"{base}.xcm"
        
comm = (f"simputfile Simput={simputfile} Src_Name=first RA=0. Dec=0. srcFlux={simput_flux} Elow=0.01 Eup=20. "
        f"NBins=1000 Emin=2 Emax=10. XSPECFile={xcmfile} clobber=yes")
#print(comm)
run_comm(comm, "Creating simputfile")

In [ ]:
# new simulation
evtfile = f"sim_evt_{base}.fits"

comm = (f"runsixt XMLFile={xml} RA=0. Dec=0. Prefix='' Simput={simputfile} EvtFile={evtfile} Exposure=1000 clobber=yes")
#print(comm)
run_comm(comm, "Simulating source with  WFI  instrument")

## 10.2.5 Step 3: Analyzing the simulation (``imgev`` and ``makespec``)

### <span style="color:blue">$\textbf{Exercise}$</span>: 
#### Take a lookat the structure of the event file. How many events were simulated? Speculate on the meaning of the individual columns in the event file 

In [ ]:
evtfile = f"sim_evt_{base}.fits"
f = fits.open(evtfile)
f.info()

# events
data = f["EVENTS"].data
nevents = data.shape[0]
print("\nNumber of simulated events:{}\n".format(nevents))

# columns 
print("Columns in event file:")
f[1].columns.names

In [ ]:
# close event file
f.close()

### Create an image (`imgev`)

In [ ]:
# create image
imgfile = f"img_{base}.fits"
        
comm = (f"imgev EvtFile={evtfile} Image={imgfile} CoordinateSystem=0 Projection=TAN NAXIS1=512 NAXIS2=512 " +
        f"CUNIT1=deg CUNIT2=deg CRVAL1=0.0 CRVAL2=0.0 CRPIX1=256.5 CRPIX2=256.5 CDELT1=-6.207043e-04 " +
        f"CDELT2=6.207043e-04 history=true clobber=yes")  
#print(comm)
run_comm(comm, "Creating WFI image")

In [ ]:
# Plot image
plt.clf()
#plt.rcParams['axes.grid'] = False # to avoid a warning about axes grids in latest versions of matplotlib
plt.rcParams['axes.grid'] = True
image_data = fits.getdata(imgfile, ext=0)
fig, _axs = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
axs = _axs.flatten()
cmap = plt.cm.PRGn

# Left plot
im = axs[0].imshow(image_data, cmap=cmap, origin="lower")
fig.colorbar(im,ax=axs[0])

# Zoomed plot
im = axs[1].imshow(image_data[210:300, 210:300], cmap=cmap, origin="lower")
fig.colorbar(im,ax=axs[1])

# Titles
axs[0].set_title("WFI chip central mcrab source")
axs[1].set_title("Central source zoom")


### Create an spectrum (`makespec`)

In [ ]:
# Create spectrum filtering the events with the condition given by "EventFilter" 
specfile = f"spec_{base}.pha"

comm = (f"makespec EvtFile={evtfile} Spectrum={specfile} clobber=yes RSPPath={xmldir} " + 
        f"EventFilter='(RA>359.95 || RA<0.05) && Dec>-0.05 && Dec<+0.05' ")
#print(comm)
run_comm(comm, "Creating spectum with WFI")

### <span style="color:blue">$\textbf{Exercise}$</span>: 
#### Obtain the names of the the response matrix and the ancilliary response matrix (the “RMF” and the “ARF”). These names can be obtained by looking at the values of the RESPFILE and ANCRFILE keywords


In [ ]:
f = fits.open(specfile)
f.info()
rmf = f[1].header['RESPFILE']
arf = f[1].header['ANCRFILE']
f.close()
print(f"Response Matrix (RMF):{rmf}")
print(f"Ancilliary Response Matrix (ARF):{arf}")

In [ ]:
#run_comm('ln -s {}'.format(rmf), 'Linking to working directory')
#run_comm('ln -s {}'.format(arf), 'Linking to working directory')

# strip path in keywords to avoid very long values and update header
rmf = os.path.basename(rmf)
arf = os.path.basename(arf)
f = fits.open(specfile, 'update')
f[1].header['RESPFILE'] = rmf
f[1].header['ANCRFILE'] = arf
f.close()

### Analyze the spectrum (XSPEC python interface)

#### 1) fit spectrum in XSPEC

In [ ]:
Plot.device = '/null'
AllData.clear()
AllModels.clear()
s0 = Spectrum(specfile)
Plot.device = "/xs"
Plot.xAxis="keV"
Plot('ldata')

#### 2) ignore energy bands where signal to noise ratio is small

In [ ]:
# As there are bins with zero variance: ignore energy bands where signal to noise ratio is small
s0.ignore("**-0.3")
s0.ignore("4.0-**")
Plot('ldata')

#### 3) Define an absorbed power law model

In [ ]:
m0 = Model("phabs*pow")

#### 4) rebin spectral data

In [ ]:
binspec = f"spec_{base}_rebin.pha"

comm = (f"grppha infile={specfile} outfile={binspec} clobber=yes comm='group min 20 & exit'")
run_comm(comm, "Rebinning spectrum")

### <span style="color:blue">$\textbf{Exercise}$</span>: 
#### Load rebinned PHA-file into XSPEC and fit an absorbed power-law to the 0.3-4 keV band


In [ ]:
binspec = f"spec_{base}_rebin.pha"
AllData.clear()
AllModels.clear()
s1 = Spectrum(binspec)
m0 = Model("phabs*pegpwrlw")
s1.ignore("**-0.3")
s1.ignore("4.0-**")
Fit.perform()

Plot("ldata")
m0.show()
Plot.device = '/null'

In [ ]:
print(f"nH fit={m0.phabs.nH.values[0]:.3f}")
print(f"Gamma fit={m0.pegpwrlw.PhoIndex.values[0]:.3f}")

#### 5) check PILEUP

In [ ]:
# check PILEUP as the cause for spectral distortion (G~1.8 instead of G~2)
evtfile = f"sim_evt_{base}.fits"
f = fits.open(evtfile)
PILEUP = f[1].data["PILEUP"]
print("The sum of PILEUP column is                  {}".format(np.sum(PILEUP)))
print("The mean of PILEUP column is                 {}".format(PILEUP.mean()))
print("The standard deviation of PILEUP column is   {}".format(PILEUP.std()))
print("The minimum of PILEUP column is              {}".format(PILEUP.min()))
print("The maximum of PILEUP column is              {}".format(PILEUP.max()))
print("The number of points used in calculation is  {}".format(len(PILEUP)))

### <span style="color:blue">$\textbf{Exercise}$</span>:  
#### change the pointing direction of the instrument away from the source in steps of 4' in right ascension and declination simultaneously. Take a look at the images of the source and the source count rate. What do you observe?

In [ ]:
plt.rcParams['axes.grid'] = True # (Set to 'False' to avoid a warning about axes grids in latest versions of matplotlib
# define values for the simulations
noffs = 3 # number of offsets (in steps of 4 arcmin)
xml = f"{xmldir}/ld_wfi_ff_large.xml"

# source is always the same
simputfile = f"{base}.fits"

# change pointing direction
plt.clf()
fig, _axs = plt.subplots(nrows=1, ncols=noffs, figsize=(20,6))
axs = _axs.flatten()
# Plotting 3 images of the source with the telescope moved away 4' and 8'
for ioff in range(0,(noffs*3),4):
    print("********************************")
    print(f"Image for offset={ioff} arcm")
    print("********************************")
    i = int(ioff/3.) 
    base_dec = f"mcrab_dec{ioff}"
    ra_off = ioff/60
    dec_off = ioff/60. #deg
    expos = 1000
    simputfile_dec = f"{base_dec}.simput"
            
    # run simulation
    evtfile_dec = f"sim_evt_{base_dec}.fits"
    comm = (f"runsixt XMLFile={xml} RA={ra_off} Dec={dec_off} Prefix='' " + 
            f"Simput={simputfile} EvtFile={evtfile_dec} Exposure={expos} clobber=yes")
    print(comm)
    #run_comm(comm, "Running simulation")
    
    # create image
    imgfile_dec = f"img_{base_dec}.fits"
    comm = (f"imgev EvtFile={evtfile_dec} Image={imgfile_dec} CoordinateSystem=0 " +
            f"Projection=TAN NAXIS1=512 NAXIS2=512 CUNIT1=deg CUNIT2=deg CRVAL1={ra_off} CRVAL2={dec_off} " + 
            f"CRPIX1=256.5 CRPIX2=256.5 CDELT1=-6.207043e-04 CDELT2=6.207043e-04")
    print(comm)
    #run_comm(comm, "Creating image")
    
    image_data = fits.getdata(imgfile_dec, ext=0)
    cmap = plt.cm.PRGn
    
    from astropy.visualization import simple_norm
    #norm = simple_norm(image_data, stretch='log', log_a=10000)
    #im = axs[i].imshow(image_data, origin="lower", cmap=cmap, norm=norm)
    #im = axs[i].imshow(image_data, cmap=cmap, origin="lower", norm=LogNorm(vmax=50))
    im = axs[i].imshow(image_data, cmap=cmap, origin="lower")
    fig.colorbar(im,ax=axs[i])
    tit = f"offset={ioff}'  // Total_cts={np.sum(image_data)}"
    axs[i].set_title(tit)


### <span style="color:blue">$\textbf{Exercise}$</span>:  
#### a) In this exercise we continue studying the effects that bright sources have on the measurement process more. Using the Crab-like spectrum defined before, increase the source flux by a factor of 10, 100, and 1000 and redo the WFI simulation. Why can you prepare the simulation using *mcrab.fits* and `fv`, and without running simputfile?



In [ ]:
# Re-using the XSPEC spectra file and thus, the SIMPUT file
simputfile = f"{base}.fits"
flux_ratio = [1, 10, 100, 1000]
nfactors = len(flux_ratio)
# colnames = f[extName].columns.names --> Use it in order to obtain the columns name to change it below

# Create new simputfiles modifying the flux from the original simputfile
for factor in flux_ratio:
    new_simputfile = f"{base}_factor{factor}.fits"
    f = fits.open(simputfile)
    initial_flux = f["SRC_CAT"].data["FLUX"]
    f["SRC_CAT"].data["FLUX"] = factor * initial_flux
    f.writeto(new_simputfile, overwrite=True)
    f.close()

In [ ]:
# do the simulation: CAREFUL!!! Simulations for FLUX=100*flux and 1000*flux take a long time to run!
xml = f"{xmldir}/ld_wfi_ff_large.xml"
expos = 1000
for i in range(nfactors):
    factor = flux_ratio[i]
    new_simputfile = f"{base}_factor{factor}.fits"
    new_evtfile = f"sim_evt_{base}_factor{factor}.fits"
    # Run simulation
    comm = (f"runsixt XMLFile={xml} RA=0. Dec=0. Prefix='' Simput={new_simputfile} " + 
            f"EvtFile={new_evtfile} Exposure={expos} clobber=yes")
    #print(comm)
    run_comm(comm, "Simulating new fluxes")

#### b) Calculate the count rates of the simulations by looking at the number of events in the file. What do you notice? Take a look at the X-ray spectrum of the 2nd brightest simulation and compare the observed photon spectrum with your input photon index. 

In [ ]:
nevents = np.zeros(nfactors)
for i in range(len(flux_ratio)):
    factor = flux_ratio[i]
    new_evtfile = f"sim_evt_{base}_factor{factor}.fits"
    # number of events in the file
    f = fits.open(new_evtfile)
    nevents[i] = f[1].header["NAXIS2"]
    print(f"Count rate for flux factor{factor}: {nevents[i]/expos} cts/s")

In [ ]:
# Create spectrum for 2nd brightest simulation (factor=100)
factor = flux_ratio[2]
new_specfile = f"spec_{base}_factor{factor}.pha"
new_evtfile = f"sim_evt_{base}_factor{factor}.fits"

comm = (f"makespec EvtFile={new_evtfile} Spectrum={new_specfile} clobber=yes RSPPath={xmldir} " + 
        f"EventFilter='(RA>359.95 || RA<0.05) && Dec>-0.05 && Dec<+0.05' ")
#print(comm)
run_comm(comm, "Creating spectum with WFI")

In [ ]:
# strip path in keywords to avoid very long values
f = fits.open(new_specfile, 'update')
f[1].header['RESPFILE'] = rmf
f[1].header['ANCRFILE'] = arf
f.close()

In [ ]:
# rebin spectrum
new_binspec = f"spec_{base}_factor{factor}_rebin.pha"
comm = (f"grppha infile={new_specfile} outfile={new_binspec} clobber=yes comm='group min 20 & exit'")    
run_comm(comm, "Rebinning spectrum")

In [ ]:
# fit power law 
Plot.device = '/null'
AllData.clear()
AllModels.clear()
s1 = Spectrum(new_binspec)
m0 = Model("phabs*pegpwrlw")
s1.ignore("**-0.3")
s1.ignore("4.0-**")
Fit.perform()

Plot("ldata")
m0.show()
Plot.device = '/null'

In [ ]:
# Get spectral index
print(f"Gamma fit={m0.pegpwrlw.PhoIndex.values[0]:.3f}")

### <span style="color:blue">$\textbf{Exercise}$</span>:  
#### To get a better feeling for what is going on, plot the value of the FITS keywords NVALID, NPVALID, NINVALID, and NPINVALI as a function of the input flux. Why is the pile up fraction NPVALID/NVALID a good measure for the scientific quality of the data?

In [ ]:
nvalid = np.zeros(nfactors)     # gives the total number of events that were classified as valid in the simulation
npvalid = np.zeros(nfactors)    # fraction of valid events caused by two or more photons (pileup)
ninvalid = np.zeros(nfactors)   # gives the total number of rejected events that were correctly classified as invalid during the simulation
npinvali = np.zeros(nfactors)   #  the fraction of these invalid events affected by pile up

for i in range(nfactors):
    factor = flux_ratio[i]
    new_evtfile = f"sim_evt_{base}_factor{factor}.fits"
    
    # Save the parameters wanted of the event file in an empty list
    f = fits.open(new_evtfile)
    nvalid[i] = (f['EVENTS'].header['NVALID'])
    npvalid[i] = (f['EVENTS'].header['NPVALID'])
    ninvalid[i] = (f['EVENTS'].header['NINVALID'])
    npinvali[i] = (f['EVENTS'].header['NPINVALI'])

    data_quality = npvalid[i]/nvalid[i]
    print("###############################################################")
    print(f"Valid events: {nvalid[i]} of which {npvalid[i]} are piled-up")
    print(f"InValid events: {ninvalid[i]} of which {npinvali[i]} are piled-up")
    print(f"Data quality={data_quality:.3f}")
    print("###############################################################")
    f.close

print(f"NVALID:   {nvalid}")
print(f"NPVALID:  {npvalid}")
print(f"NINVALID: {ninvalid}")
print(f"NPINVALI: {npinvali}")

In [ ]:
# Plot histogram of the data collected from the different simulations of the 4 fluxes.

fig, _axs = plt.subplots(nrows=1, ncols=2, figsize=(20,6))
axs = _axs.flatten()
#flux_labels = ['1', '10', '100', '1000']
flux_labels = [str(a) for a in flux_ratio]
x = np.arange(len(flux_labels))
width = 0.25
axs[0].bar(x-0.75*width, nvalid, width/2, log=True, label="valid")
axs[0].bar(x-width/4, ninvalid, width/2, log=True, label="invalid")
axs[0].bar(x+width/4, npvalid, width/2, log=True,label="piled-up valid")
axs[0].bar(x+0.75*width, npinvali, width/2, log=True,label="piled-up invalid")
axs[0].set_xlabel("Flux ratio")
axs[0].set_ylabel('Events')
axs[0].set_title('Event types for different fluxes')
axs[0].set_xticks(x)
axs[0].set_xticklabels(flux_labels)
axs[0].legend()

data_quality = (1-npvalid/nvalid)

axs[1].plot(flux_ratio, data_quality, marker='o', linestyle='--')
axs[1].set_xlabel("Flux ratio")
axs[1].set_ylabel("Data Quality")

### <span style="color:blue">$\textbf{Exercise}$</span>:  

#### Generate a SIMPUT file with a harder spectral shape (e.g., Γ = 1.5) than our example source and has the same flux. Place the source at a position that is 10" away from our source. Merge both SIMPUT files with the simputmerge tool (use `plist` to learn about the parameters of this tool!). Then run a 5 ks simulation and study how well you can separate both sources.

    

In [ ]:
# xspec file
AllModels.clear()
AllData.clear()
base_harder = "hardersrc"
xcm_harder = f"{base_harder}.xcm"
mod_harder = Model("phabs*pegpwrlw")
mod_harder.phabs.nH = 0.2
mod_harder.pegpwrlw.PhoIndex = 1.5
mod_harder.pegpwrlw.eMin = 2.
mod_harder.pegpwrlw.eMax = 10.
mod_harder.pegpwrlw.norm = 21.6 # same unabsorbed 2-10 keV flux as initial mcrab source (see Section 10.2.2)
mod_harder.show()
AllModels.calcFlux("2. 10.")
simput_flux_harder = AllModels(1).flux[0]

clobber = True
# If clobber is true and the file exists, it will be removed before creating a new one.
if os.path.exists(xcm_harder):
    if clobber:
        os.remove(xcm_harder)
    else:
        raise Exception("Xspec file ({}) already exists: it will not be overwritten".format(xcm_harder)) 
Xset.save(xcm_harder)

In [ ]:
# simputfile
simputfile10 = f"{base_harder}_10.fits"
ra = 0.0
dec10 = 10.0/3600 #deg
Elow = 0.01 # to cover full range of ARF
Eup = 20  # to cover full range of ARF
Emin = 2
Emax = 10

comm = (f"simputfile Simput={simputfile10} Src_Name=first RA={ra} Dec={dec10} srcFlux={simput_flux_harder} " + 
        f"Elow={Elow} Eup={Eup} NBins=1000 Emin={Emin} Emax={Emax} XSPECFile={xcm_harder} clobber=yes")
print(comm)
#run_comm(comm, "Creating simputfile")

In [ ]:
# merge simput files
simput_merged10 = f"{base}_{base_harder}_10_merged.fits"
comm = (f"simputmerge Infile1={simputfile} Infile2={simputfile10} Outfile={simput_merged10} clobber=yes FetchExtensions=yes")
#print(comm)
run_comm(comm, 'Merging original mcrab and newsrc_offset simput files')

In [ ]:
# run 5 ks simulation
expos=5000
xml = f"{xmldir}/ld_wfi_ff_large.xml"
evtfile_merged10 = f"sim_evt_{base}_{base_harder}_10_merged.fits"

comm = (f"runsixt XMLFile={xml} RA=0. Dec=0. Prefix='' Simput={simput_merged10} " + 
        f"EvtFile={evtfile_merged10} Exposure={expos} clobber=yes")
#print(comm)
run_comm(comm, "Simulating new fluxes")

In [ ]:
# study source separation (histogram of Dec positions) 
f = fits.open(evtfile_merged10)
data = f['EVENTS'].data
decVals = data['DEC']
fig = plt.figure(figsize=(8, 4))
ax1 = fig.add_subplot(1, 1, 1)
ax1.set_xlabel("DEC ('')")
ax1.set_ylabel("# Counts")
# create histogram
bh, bc, _ = ax1.hist(decVals[(decVals>-0.01) & (decVals<0.01)]*3600, bins=100, density=False, label="Histogram", alpha=0.5)

### <span style="color:blue">$\textbf{Exercise}$</span>:  
#### Repeat the previous exercise for a source separation of 5'' and extract spectra for both sources. Generate corrected ARFs for the sources by adjusting the parameters of the `sixte_arfgen` call above and use your favorite X-ray spectral analysis program to see how the spectral shape is affected.

In [ ]:
# simputfile for src at 5"
simputfile5 = f"{base_harder}_5.fits"
ra = 0.0
dec5 = 5.0/3600 #deg
Elow = 0.01 # to cover full range of ARF
Eup = 20  # to cover full range of ARF
Emin = 2
Emax = 10

comm = (f"simputfile Simput={simputfile5} Src_Name=first RA={ra} Dec={dec5} srcFlux={simput_flux_harder} " + 
        f"Elow={Elow} Eup={Eup} NBins=1000 Emin={Emin} Emax={Emax} XSPECFile={xcm_harder} clobber=yes")
#print(comm)
run_comm(comm, "Creating simputfile")

In [ ]:
# merge simput files
simput_merged5 = f"{base}_{base_harder}_5_merged.fits"
comm = (f"simputmerge Infile1={simputfile} Infile2={simputfile5} Outfile={simput_merged5} clobber=yes FetchExtensions=yes")
#print(comm)
run_comm(comm, 'Merging original mcrab and newsrc_offset simput files')

In [ ]:
# run 5 ks simulation
expos=5000
xml = f"{xmldir}/ld_wfi_ff_large.xml"
evtfile_merged5 = f"sim_evt_{base}_{base_harder}_5_merged.fits"

comm = (f"runsixt XMLFile={xml} RA=0. Dec=0. Prefix='' Simput={simput_merged5} " + 
        f"EvtFile={evtfile_merged5} Exposure={expos} clobber=yes")
#print(comm)
run_comm(comm, "Simulating new fluxes")

In [ ]:
# study source separation (histogram of Dec positions) 
f = fits.open(evtfile_merged5)
data = f['EVENTS'].data
decVals = data['DEC']
fig = plt.figure(figsize=(8, 4))
ax1 = fig.add_subplot(1, 1, 1)
ax1.set_xlabel("DEC ('')")
ax1.set_ylabel("# Counts")
# create histogram
bh, bc, _ = ax1.hist(decVals[(decVals>-0.005) & (decVals<0.005)]*3600, bins=100, density=False, label="Histogram", alpha=0.5)

In [ ]:
# extract images + define regions + correct ARF
radius = 5. # arcs

for off in (5, 10):
    arfcorr = f"arf_corrected_{off}.fits"
    regcircle = f"circle_{off}.reg"
    offgrad = off/3600.
    image_merged = f"img_{base}_{base_harder}_{off}_merged.fits"
    evtfile_merged = f"sim_evt_{base}_{base_harder}_{off}_merged.fits"

    # extract image
    comm = (f"imgev EvtFile={evtfile_merged} Image={image_merged} CoordinateSystem=0 " +
            f"Projection=TAN NAXIS1=512 NAXIS2=512 CUNIT1=deg CUNIT2=deg CRVAL1=0 CRVAL2=0 " + 
            f"CRPIX1=256.5 CRPIX2=256.5 CDELT1=-6.207043e-04 CDELT2=6.207043e-04")
    print(comm)
    #run_comm(comm, f"Extracting image with src at {off}")
    
    # define region
    with open(regcircle, 'w') as fw:
        fw.write("fk5\n")
        fw.write(f'circle(0.,{offgrad},{radius}")')
        
    # correct ARF
    comm = (f"sixte_arfgen ARFCorr={arfcorr} XMLFile={xml} PointingRA=0.0 PointingDec=0.0 " + 
            f"SourceRA=0.0 SourceDec={offgrad} ImageFile={image_merged} regfile={regcircle}")
    print(comm)
    #run_comm(comm,"Correcting ARF for src at {off}")


In [ ]:
# spectrum of src at 5" and 10"
radInDeg = radius/3600.  # 5" radius
for off in (5, 10):
    RAmin = 360.-radInDeg
    RAmax = 0.+radInDeg
    DECmin = off/3600 - radInDeg
    DECmax = off/3600 + radInDeg
    filt = f"'(RA>{RAmin}|| RA<{RAmax}) && Dec>{DECmin}&& Dec<+{DECmax}'"
    specfile = f"{base_harder}_{off}_spec.fits"
    evtfile_merged = f"sim_evt_{base}_{base_harder}_{off}_merged.fits"
    comm = (f"makespec EvtFile={evtfile_merged} Spectrum={specfile} clobber=yes RSPPath={xmldir} EventFilter={filt}")
    #print(comm)
    run_comm(comm, "Extracting spectrum of source at 5")

In [ ]:
# assign new ARFs in headers
for off in (5,10):
    arfcorr = f"arf_corrected_{off}.fits"
    specfile = f"{base_harder}_{off}_spec.fits"
    f = fits.open(specfile, 'update')
    rmflong = f[1].header['RESPFILE'] 
    rmf = os.path.basename(rmflong)
    f[1].header['RESPFILE'] = rmf # strip path to avoid large names (local symlink)
    f[1].header['ANCRFILE'] = arfcorr
    f.close()

In [ ]:
# rebin spectra
for off in (5, 10):
    specfile = f"{base_harder}_{off}_spec.fits"
    rebin = f"{base_harder}_{off}_spec.grp"
    comm = (f"grppha infile={specfile} outfile={rebin} clobber=yes comm='group min 20 & exit'")
    run_comm(comm, f'Rebinning spectrum of src at {off}"')

In [ ]:
# fit spectra to show variation of spectral shape 
# Remember to run the first simulation in order to ge s0 spectrum
Gammas = dict()
for off in (5, 10):
    rebin = f"{base_harder}_{off}_spec.grp"
    Plot.device = '/null'
    AllData.clear()
    AllModels.clear()
    Plot.device = "/xs"
    Plot.xAxis="KeV"
    m = Model("phabs*pow")
    s0 = Spectrum(rebin)
    s0.ignore("**-0.3")
    s0.ignore("4.0-**")

    Fit.perform()
    Plot("ldata")
    Gammas[off] = m. powerlaw.PhoIndex.values[0]
    time.sleep(5) # Add delay to separate the plots

In [ ]:
for off in (5, 10):
    print(f'Spectral Index for {off}" source: {Gammas[off]:.3f}')

### <span style="color:blue">$\textbf{Exercise}$</span>:  
#### In order to quantify the mixing of photons from both sources further, we can take a look at the diagnostic information contained in the event file. Specifically, use the FTOOL `fselect` and, using the row selection syntax, generate a new event file for one of the two sources by selecting the events in the region around the source. Determine the fraction of photons from the other source that “contaminate” the selection region by using the information in the `SRC_ID` column of the event file. For each event, this column contains information about the source in which it originated, in form of the ID of that source in the SIMPUT file.


In [ ]:
# get selected events in central source region
evt0 = "sim_evt_0.fits"
radInDeg = 5./3600.
RAmin = 360.-radInDeg
RAmax = 0.+radInDeg

DECmin = 0. - radInDeg
DECmax = 0. + radInDeg
filt = f"'(RA>{RAmin}|| RA<{RAmax}) && Dec>{DECmin}&& Dec<+{DECmax}'"
comm = (f"fselect infile={evtfile_merged5}+1 outfile={evt0} expr={filt} clobber=yes")
run_comm(comm, "Selecting events for source at centre")

# get selected events in 5" offset source region
evt5 = "sim_evt_5.fits"
DECmin = 5./3600 - radInDeg
DECmax = 5./3600 + radInDeg
filt = f"'(RA>{RAmin}|| RA<{RAmax}) && Dec>{DECmin}&& Dec<+{DECmax}'"
comm = (f"fselect infile={evtfile_merged5}+1 outfile={evt5} expr={filt} clobber=yes")
run_comm(comm, "Selecting events for source at 5")
    

In [ ]:
# Total counts (both sources)
f = fits.open(evtfile_merged5)
data = f['EVENTS'].data
print("Total number of photons in both sources (central & 5' offset)=",data.shape[0])
f.close

# Counts from central source
f = fits.open(evt0)
data = f['EVENTS'].data
idVals = data['SRC_ID']
f.close
nph = len(idVals)
nph2 =len(idVals[idVals[:,0]==2][:,0])
print(f'SRC1 (central): Number of photons from src2 (5" offset) = {nph2} ({nph2/nph*100:.2f} %)')

# Counts from 5" offset source
f = fits.open(evt5)
data = f['EVENTS'].data
idVals = data['SRC_ID']
f.close
nph = len(idVals)
nph1 =len(idVals[idVals[:,0]==1][:,0])
print(f'SRC2 (5" offset): Number of photons from src1 (central) = {nph1} ({nph1/nph*100:.2f} %)')
